In [72]:
import pandas as pd
import numpy as np
import re

In [73]:
filepath = '/Users/adamklie/Desktop/FA 2019/MED 277/project/results/mortality_unstructured_ALL_notes.csv'
notes_df = pd.read_csv(filepath, na_filter=False)

In [74]:
filepath = '/Users/adamklie/Desktop/FA 2019/MED 277/project/results/dataset.csv'
structured_df = pd.read_csv(filepath, na_filter=False)

#### Check structured data first

In [75]:
fe_patients = structured_df[structured_df['STRUCTURED_DATA_FUNGAL_ENDOPHTHALMITITS'] == 1]['SUBJECT_ID'].values
structured_df[structured_df["SUBJECT_ID"].isin(fe_patients)]

,Unnamed: 0,X,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,SPEC_TYPE_DESC,ORG_NAME,DOB,DOD,...,MARITAL_STATUS,ETHNICITY,DIAGNOSIS,INDWELLING_CATHETERS,IV_DRUG_USE,DIABETIC,RECENT_SURGERY,IMMUNOSUPRESSION,HYPERALIMENTATION,STRUCTURED_DATA_FUNGAL_ENDOPHTHALMITITS
89,90,90,12796,118237,2186-01-18 00:00:00,2186-01-18 14:37:00,FLUID RECEIVED IN BLOOD CULTURE BOTTLES,CANDIDA ALBICANS,2156-09-21 00:00:00,2186-01-23 00:00:00,...,SINGLE,WHITE,SPONTANEOUS BACTERIAL PERITONITIS,0,0,0,1,0,0,1
138,139,139,19620,183777,2168-05-19 00:00:00,2168-05-19 15:00:00,BLOOD CULTURE ( MYCO/F LYTIC BOTTLE),"CANDIDA ALBICANS, PRESUMPTIVE IDENTIFICATION",2120-09-25 00:00:00,2168-05-19 00:00:00,...,SINGLE,WHITE,SEPSIS,1,0,1,1,0,0,1


In [76]:
structured_notes_df = notes_df[notes_df["SUBJECT_ID"].isin(fe_patients)]
structured_notes_df.shape

(45, 28)

In [77]:
# Regular expressions to check
fungal_endo = 'fungal endophthalmitis'
fungal_endo_misspelling = 'fungal endopthalmitis'
fungal_ocular_involvement = 'fungal ocular involvement'
full = 'ophth|opth|eye|ocul|retina|fundus|dilat|endophthalmitis|chorioretinitis|vitritis'
full_minus = 'ophth|opth|ocul|retina|fundus|endophthalmitis|chorioretinitis|vitritis'

In [78]:
regex = re.compile(full_minus)
pos_table = {}
for index, row in structured_notes_df.iterrows():
    note_text = row['TEXT']
    is_match = regex.search(note_text)
    if is_match != None:
        subject_id = row['SUBJECT_ID']
        text = note_text[is_match.start()-5:is_match.end()+5]
        pos_table.setdefault(subject_id, '')
        pos_table[subject_id] = pos_table[subject_id] + text

In [79]:
pos_table

{19620: 'neuroophthalmol'}

#### Check notes for regular expressions

In [104]:
notes = notes_df['TEXT']
fe_mask = notes.str.contains(full_minus, regex = True)
notes_df[fe_mask].shape

(683, 28)

In [100]:
%%time
regex = re.compile(full_minus)
pos_table = {}
note_dict = {}
for index, row in notes_df.iterrows():
    note_text = row['TEXT']
    is_match = regex.search(note_text)
    if is_match != None:
        subject_id = row['SUBJECT_ID']
        text = note_text[is_match.start()-100:is_match.end()+100]
        pos_table.setdefault(subject_id, []).append(text)
        note_dict.setdefault(subject_id, '')
        note_dict[subject_id] = note_dict[subject_id] + note_text + '\n'

CPU times: user 3.3 s, sys: 13.1 ms, total: 3.31 s
Wall time: 3.35 s


In [101]:
surrounding_text_df = pd.DataFrame(pos_table.values(), index = pos_table.keys())
note_dict_df = pd.DataFrame(note_dict.values(), index = note_dict.keys())

In [103]:
note_dict_df.to_csv('../results/full_minus_notes.csv')